# Final Project code 
### Target : Which college area is best to setup restaurant and which area is best for marketing the buissness

In [3]:
import requests  # library to handle requests
import pandas as pd  # library for data analsysis
import numpy as np  # library to handle data in a vectorized manner
import random  # library for random number generation

# module to convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

# libraries for displaying images
from IPython.display import Image
from IPython.core.display import HTML

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
#!conda install -c conda-forge folium=0.5.0 --yes 

import folium  # plotting library

print('Folium installed')
print('Libraries imported.')


Folium installed
Libraries imported.


In [4]:
wiki_link = 'https://en.wikipedia.org/wiki/List_of_educational_institutions_in_Pune'


## Using FourSquare API Call

In [5]:
CLIENT_ID ='WYLCMMICTAG4ZKF1JQ2HZBRLVNJWBKVL1ZJ1YBNG0A2W14F2'
CLIENT_SECRET = 'AAFOAZR4ZI210CI1FPFEIRHU3IR12PGR00CQ3GDID3YCF0DE'
VERSION ='20200718'
LIMIT =30

In [6]:
#MIT WPU
geolocator = Nominatim(user_agent='CourseraCapstoneProject')
location = geolocator.geocode("Maharashtra Institute of Technology").raw
lat = location['lat']
lon = location['lon']
print("Latitude: ", lat)
print("Longitude: ", lon)

Latitude:  18.518667899999997
Longitude:  73.81556127173374


In [7]:
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    lat,
    lon,
    radius,
    100)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=WYLCMMICTAG4ZKF1JQ2HZBRLVNJWBKVL1ZJ1YBNG0A2W14F2&client_secret=AAFOAZR4ZI210CI1FPFEIRHU3IR12PGR00CQ3GDID3YCF0DE&v=20200718&ll=18.518667899999997,73.81556127173374&radius=1000&limit=100'

In [8]:
results = requests.get(url).json()
results


{'meta': {'code': 200, 'requestId': '5f26bad3a52b5d02f7e74deb'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Pune',
  'headerFullLocation': 'Pune',
  'headerLocationGranularity': 'city',
  'totalResults': 19,
  'suggestedBounds': {'ne': {'lat': 18.527667909000005,
    'lng': 73.82503502555852},
   'sw': {'lat': 18.50966789099999, 'lng': 73.80608751790896}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8c891d0aaf7fe613fbcd53',
       'name': 'ARAI Hilltop',
       'location': {'lat': 18.51973678124073,
        'lng': 73.81767671579341,
        'labeledLatLngs': [{'label': 'display',
          'lat': 18.51973678124073,
          'lng': 73.81767671579341}],
        'distance'

In [9]:
#for extracting venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues)  # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories',
                    'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(
    get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
print(nearby_venues.shape)
nearby_venues

(19, 4)


,name,categories,lat,lng
0,ARAI Hilltop,Other Great Outdoors,18.519737,73.817677
1,Vetal Tekdi,Trail,18.522266,73.818349
2,Burger Barn Café,Burger Joint,18.510960,73.815502
3,IDDOS,South Indian Restaurant,18.510441,73.816445
4,Cafe Coffee Day,Coffee Shop,18.513672,73.815463
5,Richie Rich,Sandwich Place,18.514001,73.815417
6,Hotel Durga,Coffee Shop,18.510384,73.816282
7,Café Creme,Coffee Shop,18.509929,73.816072
8,Sharao Lawns and Restaurant,Beer Garden,18.512417,73.812991
9,P. N. Gadgil Jewllers,Jewelry Store,18.510374,73.819083


In [11]:
nearby_venues.categories.unique()

array(['Other Great Outdoors', 'Trail', 'Burger Joint',
       'South Indian Restaurant', 'Coffee Shop', 'Sandwich Place',
       'Beer Garden', 'Jewelry Store', 'Indian Restaurant',
       'Chinese Restaurant', 'Convenience Store', 'Fast Food Restaurant',
       'Restaurant', 'Food', 'Airport', 'Café'], dtype=object)

### Listing the institutes which we would study

In [12]:
insts = ['Maharashtra Institute of Technology','Film and Television Institute of India','ILS Law College','Fergussson College','College of Engineering, Pune',
         'AISSMS College of Hotel Management','Modern College of Engineering','Bhandarkar Oriental Research Institute','Abasaheb Garware College','Armed Forces Medical College','Symbiosis Institute of Management Studies (SIMS)',
        'NCCS research institute','Institute of Management and Career Courses','College of Agricultural Banking',]
print(len(insts))


14


In [13]:
insts = [x+" Pune" for x in insts]
insts

['Maharashtra Institute of Technology Pune',
 'Film and Television Institute of India Pune',
 'ILS Law College Pune',
 'Fergussson College Pune',
 'College of Engineering, Pune Pune',
 'AISSMS College of Hotel Management Pune',
 'Modern College of Engineering Pune',
 'Bhandarkar Oriental Research Institute Pune',
 'Abasaheb Garware College Pune',
 'Armed Forces Medical College Pune',
 'Symbiosis Institute of Management Studies (SIMS) Pune',
 'NCCS research institute Pune',
 'Institute of Management and Career Courses Pune',
 'College of Agricultural Banking Pune']

### Function to get the respective latitudes and longitudes 

In [14]:
def coords(institute):
    d = {}
    d['institute'] = institute
    geolocator = Nominatim(user_agent='CourseraCapstoneProject')
    try:
        location = geolocator.geocode(institute).raw
        d['latitude'] = location['lat']
        d['longitude'] = location['lon']
        return d
    except Exception as e:
        print("Institute %s not found"%institute)
        return -1


In [15]:


l = []
for i in insts:
    details = coords(i)
    if(details!=-1):
        l.append(coords(i))
print(l)
print(len(l))

[{'institute': 'Maharashtra Institute of Technology Pune', 'latitude': '18.518667899999997', 'longitude': '73.81556127173374'}, {'institute': 'Film and Television Institute of India Pune', 'latitude': '18.51330805', 'longitude': '73.82673634875577'}, {'institute': 'ILS Law College Pune', 'latitude': '18.51683155', 'longitude': '73.8279306824409'}, {'institute': 'Fergussson College Pune', 'latitude': '18.524148500000003', 'longitude': '73.8385914880175'}, {'institute': 'College of Engineering, Pune Pune', 'latitude': '18.5338842', 'longitude': '73.88056288653043'}, {'institute': 'AISSMS College of Hotel Management Pune', 'latitude': '18.5268898', 'longitude': '73.85438727298441'}, {'institute': 'Modern College of Engineering Pune', 'latitude': '18.52546885', 'longitude': '73.84603170103503'}, {'institute': 'Bhandarkar Oriental Research Institute Pune', 'latitude': '18.519283100000003', 'longitude': '73.82925831715562'}, {'institute': 'Abasaheb Garware College Pune', 'latitude': '18.5117

In [16]:
#Pune location
pune_lat = 18.6452
pune_lon = 73.9231

In [17]:
# Plotting the institutes on map 
inst_map = folium.Map(location = [pune_lat, pune_lon], zoom_start=11)

for d in l:
    folium.CircleMarker(
    [float(d['latitude']), float(d['longitude'])],
        radius = 5, 
        popup = d['institute'],
        fill = True,
        color = '#0012EE',
        fill_color = 'red',
        fill_opacity = 0.5
    ).add_to(inst_map)
    
inst_map


### Cleaning on basis of food categories

In [18]:
food_cats = ['Ice Cream Shop', 'Sandwich Place', 'Food Truck', 'Fast Food Restaurant', 'Indian Restaurant', 'Steakhouse',
            'Chinese Restaurant', 'Coffee Shop', 'Bakery', 'Café', 'Middle Eastern Restuarant']

In [19]:
print(l[0])
inst1 = l[0]['institute']
lat = l[0]['latitude']
lon = l[0]['longitude']

{'institute': 'Maharashtra Institute of Technology Pune', 'latitude': '18.518667899999997', 'longitude': '73.81556127173374'}


In [20]:
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    lat,
    lon,
    radius,
    100)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=WYLCMMICTAG4ZKF1JQ2HZBRLVNJWBKVL1ZJ1YBNG0A2W14F2&client_secret=AAFOAZR4ZI210CI1FPFEIRHU3IR12PGR00CQ3GDID3YCF0DE&v=20200718&ll=18.518667899999997,73.81556127173374&radius=1000&limit=100'

In [48]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f26715a932b2b742013da88'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Pune',
  'headerFullLocation': 'Pune',
  'headerLocationGranularity': 'city',
  'totalResults': 19,
  'suggestedBounds': {'ne': {'lat': 18.527667909000005,
    'lng': 73.82503502555852},
   'sw': {'lat': 18.50966789099999, 'lng': 73.80608751790896}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8c891d0aaf7fe613fbcd53',
       'name': 'ARAI Hilltop',
       'location': {'lat': 18.51973678124073,
        'lng': 73.81767671579341,
        'labeledLatLngs': [{'label': 'display',
          'lat': 18.51973678124073,
          'lng': 73.81767671579341}],
        'distance'

### Filtering food related venues

In [21]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues)  # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories',
                    'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(
    get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues = nearby_venues[nearby_venues['categories'].isin(food_cats)]
print(nearby_venues.shape)
nearby_venues

(9, 4)


,name,categories,lat,lng
4,Cafe Coffee Day,Coffee Shop,18.513672,73.815463
5,Richie Rich,Sandwich Place,18.514001,73.815417
6,Hotel Durga,Coffee Shop,18.510384,73.816282
7,Café Creme,Coffee Shop,18.509929,73.816072
10,Spice Garden,Indian Restaurant,18.511325,73.813812
11,Hot Sip,Chinese Restaurant,18.510265,73.815908
13,Maggi Station,Fast Food Restaurant,18.513899,73.815494
17,Chaitanya Garden,Indian Restaurant,18.510123,73.815454
18,#Adda,Café,18.510345,73.818410


In [22]:
venues_i1 = []
for index, row in nearby_venues.iterrows():
    d = {}
    d['name'] = row['name']
    d['lat'] = row['lat']
    d['lng'] = row['lng']
    venues_i1.append(d)


### Plotting nearby eateries to MIT

In [23]:
inst_map = folium.Map(location=[float(lat), float(lon)], zoom_start=15)

for d in venues_i1:
    folium.CircleMarker(
        [float(d['lat']), float(d['lng'])],
        radius=5,
        popup=d['name'],
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(inst_map)

folium.CircleMarker(
    [float(lat), float(lon)],
    radius=7.5,
    popup=inst1,
    fill=True,
    color='red',
    fill_color='red'
).add_to(inst_map)
inst_map

## Now we will integrate all the selected institutes 

In [25]:
def coords(institute):
    d = {}
    d['institute'] = institute
    geolocator = Nominatim(user_agent='myapplication')
    try:
        location = geolocator.geocode(institute).raw
        d['latitude'] = location['lat']
        d['longitude'] = location['lon']
        return d
    except Exception as e:
        print("Institute %s not found"%institute)
        return -1


In [26]:
institutes = []
for i in insts:
    details = coords(i)
    if(details!=-1):
        institutes.append(coords(i))
print(institutes)
print(len(institutes))

[{'institute': 'Maharashtra Institute of Technology Pune', 'latitude': '18.518667899999997', 'longitude': '73.81556127173374'}, {'institute': 'Film and Television Institute of India Pune', 'latitude': '18.51330805', 'longitude': '73.82673634875577'}, {'institute': 'ILS Law College Pune', 'latitude': '18.51683155', 'longitude': '73.8279306824409'}, {'institute': 'Fergussson College Pune', 'latitude': '18.524148500000003', 'longitude': '73.8385914880175'}, {'institute': 'College of Engineering, Pune Pune', 'latitude': '18.5338842', 'longitude': '73.88056288653043'}, {'institute': 'AISSMS College of Hotel Management Pune', 'latitude': '18.5268898', 'longitude': '73.85438727298441'}, {'institute': 'Modern College of Engineering Pune', 'latitude': '18.52546885', 'longitude': '73.84603170103503'}, {'institute': 'Bhandarkar Oriental Research Institute Pune', 'latitude': '18.519283100000003', 'longitude': '73.82925831715562'}, {'institute': 'Abasaheb Garware College Pune', 'latitude': '18.5117

### Plotting all the institutes that we gonna consider

In [28]:
import string
inst_map = folium.Map(location = [pune_lat, pune_lon], zoom_start=11, tiles = "Stamen Terrain")

for d in institutes:
    folium.CircleMarker(
    [float(d['latitude']), float(d['longitude'])],
        radius = 5, 
        popup = d['institute'].translate(str.maketrans('', '', string.punctuation)),
        fill = True,
        color = '#0012EE',
        fill_color = 'red',
        fill_opacity = 0.5
    ).add_to(inst_map)
    
inst_map


### Using foursquare api again for food categories across all the institutes in Pune

In [31]:
url = 'https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,)

results = requests.get(url).json()

In [32]:
food_categs = []
for i in results['response']['categories'][3]['categories']:
    food_categs.append(i['name'])
print(len(food_categs))

92


In [34]:
food_categs[:-1]

['Afghan Restaurant',
 'African Restaurant',
 'American Restaurant',
 'Asian Restaurant',
 'Australian Restaurant',
 'Austrian Restaurant',
 'BBQ Joint',
 'Bagel Shop',
 'Bakery',
 'Bangladeshi Restaurant',
 'Belgian Restaurant',
 'Bistro',
 'Breakfast Spot',
 'Bubble Tea Shop',
 'Buffet',
 'Burger Joint',
 'Cafeteria',
 'Café',
 'Cajun / Creole Restaurant',
 'Caribbean Restaurant',
 'Caucasian Restaurant',
 'Coffee Shop',
 'Comfort Food Restaurant',
 'Creperie',
 'Czech Restaurant',
 'Deli / Bodega',
 'Dessert Shop',
 'Diner',
 'Donut Shop',
 'Dumpling Restaurant',
 'Dutch Restaurant',
 'Eastern European Restaurant',
 'English Restaurant',
 'Falafel Restaurant',
 'Fast Food Restaurant',
 'Fish & Chips Shop',
 'Fondue Restaurant',
 'Food Court',
 'Food Stand',
 'Food Truck',
 'French Restaurant',
 'Fried Chicken Joint',
 'Friterie',
 'Gastropub',
 'German Restaurant',
 'Gluten-free Restaurant',
 'Greek Restaurant',
 'Halal Restaurant',
 'Hawaiian Restaurant',
 'Hot Dog Joint',
 'Hungar

### Function to generate a random color hex code (for use in map making):

In [35]:
import random
def randomcol():
    r = lambda: random.randint(0,255)
    return('#%02X%02X%02X' % (r(),r(),r()))

### Function to extract the category of a venue from a dataframe

In [36]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


### Main plotting function for all institutes and all the food venues within 1000 metres

In [37]:
main_map = folium.Map(location=[pune_lat, pune_lon], zoom_start=11, tiles = "Stamen Terrain")
radius = 1000

eatery_data = []

def fullplot():
    for i in institutes:
        name = i['institute']
        print(name, end=' ')
        lat = i['latitude']
        lon = i['longitude']
        
        # Using the foursquare venues API to find nearby venues for an institute
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lon,
            radius,
            100)
        results = requests.get(url).json()
        
        venues = results['response']['groups'][0]['items']

        nearby_venues = json_normalize(venues)  # flatten JSON

        # filter columns
        filtered_columns = ['venue.name', 'venue.categories',
                            'venue.location.lat', 'venue.location.lng']
        nearby_venues = nearby_venues.loc[:, filtered_columns]
        # filter the category for each row
        nearby_venues['venue.categories'] = nearby_venues.apply(
            get_category_type, axis=1)

        # clean columns
        nearby_venues.columns = [col.split(".")[-1]
                                 for col in nearby_venues.columns]
        nearby_venues = nearby_venues[nearby_venues['categories'].isin(
            food_categs)]
        
        print(", Venues: ",nearby_venues.shape[0])

        venues_i = []
        for index, row in nearby_venues.iterrows():
            d = {}
            d['name'] = row['name'].translate(str.maketrans('', '', string.punctuation))
            d['lat'] = row['lat']
            d['lng'] = row['lng']
            venues_i.append(d)
        
        # Generating a random color
        color = randomcol()
        
        # Plotting venues
        for d in venues_i:
            folium.CircleMarker(
                [float(d['lat']), float(d['lng'])],
                radius=1.5,
                popup=d['name'].translate(str.maketrans('', '', string.punctuation)),
                fill=True,
                color=color,
                fill_color='blue',
                fill_opacity=0.5
            ).add_to(main_map)
            eatery_data.append(d)
            
             # Plotting institute
        folium.CircleMarker(
            [float(lat), float(lon)],
            radius=5,
            popup=name.translate(str.maketrans('', '', string.punctuation)),
            fill=True,
            color=color,
            fill_color='red'
        ).add_to(main_map)
        
# Calling the function that adds markers to the map
fullplot()

# Printing our map
main_map

Maharashtra Institute of Technology Pune , Venues:  10
Film and Television Institute of India Pune , Venues:  34
ILS Law College Pune , Venues:  28
Fergussson College Pune , Venues:  69
College of Engineering, Pune Pune , Venues:  38
AISSMS College of Hotel Management Pune , Venues:  12
Modern College of Engineering Pune , Venues:  61
Bhandarkar Oriental Research Institute Pune , Venues:  22
Abasaheb Garware College Pune , Venues:  61
Armed Forces Medical College Pune , Venues:  5
Symbiosis Institute of Management Studies (SIMS) Pune , Venues:  2
NCCS research institute Pune , Venues:  7
Institute of Management and Career Courses Pune , Venues:  42
College of Agricultural Banking Pune , Venues:  32


### Lets use clustering algorithm to make the map more interactive

### Institute Locations using pandas dataframes

In [38]:
instdf = pd.DataFrame(institutes)
instdf.head(10)

,institute,latitude,longitude
0,Maharashtra Institute of Technology Pune,18.518667899999997,73.81556127173374
1,Film and Television Institute of India Pune,18.51330805,73.82673634875577
2,ILS Law College Pune,18.51683155,73.8279306824409
3,Fergussson College Pune,18.524148500000003,73.8385914880175
4,"College of Engineering, Pune Pune",18.5338842,73.88056288653043
5,AISSMS College of Hotel Management Pune,18.5268898,73.85438727298441
6,Modern College of Engineering Pune,18.52546885,73.84603170103503
7,Bhandarkar Oriental Research Institute Pune,18.519283100000003,73.82925831715562
8,Abasaheb Garware College Pune,18.511730399999998,73.83885371291771
9,Armed Forces Medical College Pune,18.502857499999998,73.89337950922894


### Eateries dataframe

In [39]:
eaterydf = pd.DataFrame(eatery_data)
print(len(eaterydf))
eaterydf.head(10)

423


,lat,lng,name
0,18.510960,73.815502,Burger Barn Café
1,18.513672,73.815463,Cafe Coffee Day
2,18.514001,73.815417,Richie Rich
3,18.510384,73.816282,Hotel Durga
4,18.509929,73.816072,Café Creme
5,18.511325,73.813812,Spice Garden
6,18.513899,73.815494,Maggi Station
7,18.509888,73.816110,Harsh
8,18.510123,73.815454,Chaitanya Garden
9,18.510345,73.818410,Adda


### Clustering of institutes first

In [41]:
from sklearn.cluster import KMeans
num_ci = 6
kmeans_inst = KMeans(n_clusters=num_ci, random_state=0).fit(instdf.loc[:, ['latitude', 'longitude']])
id_label_inst = kmeans_inst.labels_

In [42]:
map2 = folium.Map(location=[pune_lat, pune_lon], zoom_start=11, tiles="Stamen Terrain")
for i in range(num_ci):
    cluster = np.where(id_label_inst == i)[0]
    col = randomcol()
    for la, lo, name in zip(instdf.latitude[cluster].values, instdf.longitude[cluster].values, instdf.institute[cluster].values):
        folium.CircleMarker(
            [float(la), float(lo)],
            radius=5,
            popup=name.translate(str.maketrans('', '', string.punctuation)),
            fill=True,
            color=col,
            fill_color='red'
        ).add_to(map2)

map2

### Clustering the eateries now

In [43]:

num_ce = 10
kmeans_eat = KMeans(n_clusters=num_ce, random_state=0).fit(eaterydf.loc[:, ['lat', 'lng']])
id_label_eat = kmeans_eat.labels_

In [44]:
map3 = folium.Map(location=[pune_lat, pune_lon], zoom_start=11, tiles = "Stamen Terrain")

for i in range(num_ci):
    cluster = np.where(id_label_inst == i)[0]
    col = randomcol()
    for la, lo, name in zip(instdf.latitude[cluster].values, instdf.longitude[cluster].values, instdf.institute[cluster].values):
        folium.CircleMarker(
            [float(la), float(lo)],
            radius=5,
            popup=name.translate(str.maketrans('', '', string.punctuation)),
            fill=True,
            color=col,
            fill_color='red'
        ).add_to(map3)

for i in range(num_ce):
    cluster = np.where(id_label_eat == i)[0]
    col = randomcol()
    for la, lo, name in zip(eaterydf.lat[cluster].values, eaterydf.lng[cluster].values, eaterydf.name[cluster].values):
        folium.CircleMarker(
            [float(la), float(lo)],
            radius=1,
            popup=str(i)+ ":"+ name.translate(str.maketrans('', '', string.punctuation)),
            fill=True,
            color=col,
            fill_color='blue',
            fill_opacity=0.5
        ).add_to(map3)

map3

### Analysis 

In [49]:
from itertools import groupby

counts = [(i, len(list(c))) for i,c in groupby(sorted(id_label_eat))]
print(counts)
print(instdf['institute'])

[(0, 70), (1, 27), (2, 33), (3, 100), (4, 5), (5, 21), (6, 104), (7, 22), (8, 30), (9, 11)]
0              Maharashtra Institute of Technology Pune
1           Film and Television Institute of India Pune
2                                  ILS Law College Pune
3                               Fergussson College Pune
4                     College of Engineering, Pune Pune
5               AISSMS College of Hotel Management Pune
6                    Modern College of Engineering Pune
7           Bhandarkar Oriental Research Institute Pune
8                         Abasaheb Garware College Pune
9                     Armed Forces Medical College Pune
10    Symbiosis Institute of Management Studies (SIM...
11                         NCCS research institute Pune
12      Institute of Management and Career Courses Pune
13                 College of Agricultural Banking Pune
Name: institute, dtype: object


## Result : Analysis and Observations 

Cluster 4:  **College  of Engineering Pune** has very few eateries around it. It is a well reputed college in Pune,infact the top college and has a large student population.

A good eatery aimed at students, or even at the general public as it connect to the most premium sector of Pune that is JM Road. Thereby,would do well here due to sheer lack of competition in the vicinity.

### Hence we find that the best place for a food shop aimed at student crowd in general will be near College of Engineering COEP, ShivajiNagar,Pune

### 2 nd result 

We also find that 3 clusters have more than 50 eateries near them.

These could be great localities to **advertise** an upcoming new fast food shop or restaurant.

That is :
1. **Maharashtra Institute of Technology ,Kothrud, Pune**
2. **Fergussson College, FC Road, Pune**
3. **Modern College of Engineering,Model Colony, Pune**

Thereby this area would be the best sectors for marketing as it consists of the most cream crowd well established colleges and restuarant there by will be helpful to attract new customers.

### Lat 